
# 🧭 Lesson 1: Breadth-First Search (BFS) and Depth-First Search (DFS)

### 📘 Artificial Intelligence — Master's Degree (LM-32)
**Instructor:** PhD Student Emanuele Nardone  
**Topic:** Uninformed Search — Traversing State Spaces

---

In this lab, we study and implement **BFS** and **DFS**, two fundamental search strategies in Artificial Intelligence.  
They are used to **explore state spaces** where no heuristic knowledge is available — that is, the search has no idea which direction is “better” and must explore all possibilities systematically.



## 🔍 Search and Problem Solving in AI

Many AI problems — from **robot navigation** to **puzzle solving** — can be modeled as *search problems*.

A search problem consists of:
- **Initial State** — where we start (e.g., robot’s position)
- **Actions** — moves possible from a state (e.g., up, down, left, right)
- **Transition Model** — defines how actions change the state
- **Goal Test** — determines if we reached the solution
- **Path Cost** — evaluates the quality of the solution (distance, time, etc.)

### Example
- **Maze Solving:**  
  - States → grid positions  
  - Actions → move up/down/left/right  
  - Goal → reach target cell  
  - Path cost → number of moves  

Uninformed searches like BFS and DFS don’t know where the goal is — they explore *blindly* until it is found.


## 🌍 Real-World Applications

### BFS Applications
1. **Social Networks** — Finding shortest connection path between two people (e.g., LinkedIn connections)
2. **GPS Navigation** — Finding shortest route between locations (when all roads have equal cost)
3. **Web Crawlers** — Systematically exploring websites level by level
4. **Network Broadcasting** — Sending packets to all nodes in a network efficiently
5. **Puzzle Solving** — Finding shortest solution to puzzles like Rubik's Cube or sliding puzzles

### DFS Applications
1. **Maze Generation** — Creating random mazes by exploring paths deeply
2. **Topological Sorting** — Ordering tasks with dependencies (e.g., course prerequisites)
3. **Cycle Detection** — Finding loops in graphs (e.g., detecting deadlocks)
4. **Pathfinding in Games** — Exploring game trees in chess, checkers
5. **File System Traversal** — Recursively exploring directories and subdirectories


## ⚙️ Breadth-First vs Depth-First Search

### 🌀 Breadth-First Search (BFS)
- Think of **BFS** like throwing a wave from the start node — it spreads equally in all directions.
- Uses a **Queue (FIFO)** structure.
- Explores nodes *by distance* from the start.
- **Complete:** Yes (if finite branching factor).
- **Optimal:** Yes, if all edges have same cost.

📦 **Analogy:** “Explore all rooms on the current floor before going upstairs.”

#### 🧮 Intuitive Pseudocode for BFS
```
1. Create an empty queue (FIFO).
2. Enqueue the start node.
3. While the queue is not empty:
      a. Dequeue the first node.
      b. If it's the goal, stop.
      c. Otherwise, enqueue all its unvisited neighbors.
```

---

### 🔁 Depth-First Search (DFS)
- **DFS** goes deep before it goes wide.
- Uses a **Stack (LIFO)** or recursion.
- **Not guaranteed** to find shortest paths.
- **Memory-efficient** for large/deep spaces.

📦 **Analogy:** “Follow one corridor to the end before checking others.”

#### 🧮 Intuitive Pseudocode for DFS
```
1. Create an empty stack (LIFO).
2. Push the start node.
3. While the stack is not empty:
      a. Pop the top node.
      b. If it's the goal, stop.
      c. Otherwise, push all its unvisited neighbors.
```


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque
import time
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
# Create a binary tree for demonstration
G = nx.DiGraph()  # Using directed graph for tree structure
edges = [
    ('A', 'B'), ('A', 'C'),      # Level 1: A has children B and C
    ('B', 'D'), ('B', 'E'),      # Level 2: B has children D and E
    ('C', 'F'), ('C', 'G'),      # Level 2: C has children F and G
    ('D', 'H'), ('D', 'I'),      # Level 3: D has children H and I
]
G.add_edges_from(edges)

# Create hierarchical layout for binary tree
def hierarchy_pos(G, root, width=1., vert_gap=0.2, vert_loc=0, xcenter=0.5):
    """Create a hierarchical layout for a tree"""
    pos = {root: (xcenter, vert_loc)}
    neighbors = list(G.neighbors(root))
    
    if len(neighbors) != 0:
        dx = width / len(neighbors)
        nextx = xcenter - width/2 - dx/2
        for neighbor in neighbors:
            nextx += dx
            pos.update(hierarchy_pos(G, neighbor, width=dx, vert_gap=vert_gap,
                                    vert_loc=vert_loc-vert_gap, xcenter=nextx))
    return pos

pos = hierarchy_pos(G, 'A')

# Visualize the binary tree
plt.figure(figsize=(10, 8))
nx.draw_networkx_nodes(G, pos, node_color='skyblue', node_size=3000, 
                      edgecolors='black', linewidths=2)
nx.draw_networkx_edges(G, pos, edge_color='gray', width=3, 
                      arrows=True, arrowsize=20, arrowstyle='->', 
                      connectionstyle='arc3,rad=0')
nx.draw_networkx_labels(G, pos, font_size=16, font_weight='bold', font_color='black')

# Add level labels
for i, (level, y) in enumerate([('Root', 0.95), ('Level 1', 0.68), ('Level 2', 0.4), ('Level 3', 0.12)]):
    plt.text(0.02, y, level, transform=plt.gca().transAxes, 
            fontsize=11, fontweight='bold', style='italic',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='lightyellow', 
                     edgecolor='black', alpha=0.7))

plt.title("Binary Tree for BFS and DFS Demonstration", fontsize=18, fontweight='bold', pad=20)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
def bfs_basic(graph, start):
    visited = set()
    queue = deque([start])
    order = []

    while queue:
        node = queue.popleft()
        if node not in visited:
            visited.add(node)
            order.append(node)
            queue.extend(neigh for neigh in graph.neighbors(node) if neigh not in visited)
    return order

print("Basic BFS order:", bfs_basic(G, 'A'))

# Visualize BFS step-by-step
def bfs_visualization(graph, pos, start):
    """Visualize BFS exploration step by step"""
    visited = set()
    queue = deque([start])
    order = []
    steps = []  # Store each step for visualization
    
    # Initial state
    steps.append({
        'current': None,
        'queue': list(queue),
        'visited': set(),
        'order': []
    })
    
    while queue:
        node = queue.popleft()
        if node not in visited:
            visited.add(node)
            order.append(node)
            neighbors = [n for n in graph.neighbors(node) if n not in visited]
            queue.extend(neighbors)
            
            # Record this step
            steps.append({
                'current': node,
                'queue': list(queue),
                'visited': visited.copy(),
                'order': order.copy(),
                'neighbors': neighbors
            })
    
    # Create visualization
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for idx, step in enumerate(steps[:8]):  # Show first 8 steps
        ax = axes[idx]
        
        # Draw edges
        nx.draw_networkx_edges(graph, pos, edge_color='lightgray', width=2, ax=ax)
        
        # Color nodes based on state
        node_colors = []
        for node in graph.nodes():
            if node == step['current']:
                node_colors.append('#FF6B6B')  # Red = currently processing
            elif node in step['visited']:
                node_colors.append('#51CF66')  # Green = visited
            elif node in step.get('queue', []):
                node_colors.append('#FFD93D')  # Yellow = in queue
            else:
                node_colors.append('#E0E0E0')  # Gray = unvisited
        
        nx.draw_networkx_nodes(graph, pos, node_color=node_colors, 
                              node_size=2000, ax=ax)
        nx.draw_networkx_labels(graph, pos, font_size=10, font_weight='bold', ax=ax)
        
        # Title showing state
        title = f"Step {idx}"
        if step['current']:
            title += f"\nProcessing: {step['current']}"
            title += f"\nQueue: {list(step['queue'])}"
        else:
            title += "\nInitial State"
        ax.set_title(title, fontsize=10, fontweight='bold')
        ax.axis('off')
    
    # Create custom legend with colored boxes
    from matplotlib.patches import Rectangle
    legend_elements = [
        Rectangle((0, 0), 1, 1, fc='#FF6B6B', edgecolor='black', label='Current'),
        Rectangle((0, 0), 1, 1, fc='#51CF66', edgecolor='black', label='Visited'),
        Rectangle((0, 0), 1, 1, fc='#FFD93D', edgecolor='black', label='In Queue'),
        Rectangle((0, 0), 1, 1, fc='#E0E0E0', edgecolor='black', label='Unvisited')
    ]
    fig.legend(handles=legend_elements, loc='upper center', ncol=4, 
              bbox_to_anchor=(0.5, 0.98), fontsize=12, frameon=True, 
              fancybox=True, shadow=True)
    
    plt.suptitle("BFS Step-by-Step Visualization", 
                 fontsize=14, fontweight='bold', y=0.92)
    plt.tight_layout()
    plt.subplots_adjust(top=0.90)
    plt.show()

bfs_visualization(G, pos, 'A')

In [ ]:
def bfs_with_path(graph, start, goal):
    """BFS that returns the shortest path from start to goal"""
    if start == goal:
        return [start]
    
    visited = set([start])
    queue = deque([[start]])  # Queue stores paths, not just nodes
    
    while queue:
        path = queue.popleft()
        node = path[-1]
        
        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = path + [neighbor]
                
                if neighbor == goal:
                    return new_path
                
                queue.append(new_path)
    
    return None  # No path found

# Find shortest path from A to I
path = bfs_with_path(G, 'A', 'I')
print(f"Shortest path from A to I: {' → '.join(path)}")
print(f"Path length: {len(path) - 1} edges")

# Visualize the shortest path
def visualize_path(graph, pos, path, title):
    """Visualize a specific path on the graph"""
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Draw all edges in light gray
    nx.draw_networkx_edges(graph, pos, edge_color='lightgray', width=1, alpha=0.3, ax=ax)
    
    # Highlight path edges
    path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
    nx.draw_networkx_edges(graph, pos, edgelist=path_edges, 
                          edge_color='#FF6B6B', width=4, ax=ax)
    
    # Color nodes
    node_colors = []
    for node in graph.nodes():
        if node == path[0]:
            node_colors.append('#51CF66')  # Green = start
        elif node == path[-1]:
            node_colors.append('#FF6B6B')  # Red = goal
        elif node in path:
            node_colors.append('#FFD93D')  # Yellow = on path
        else:
            node_colors.append('#E0E0E0')  # Gray = not on path
    
    nx.draw_networkx_nodes(graph, pos, node_color=node_colors, 
                          node_size=2500, ax=ax)
    nx.draw_networkx_labels(graph, pos, font_size=14, font_weight='bold', ax=ax)
    
    # Add step numbers on path nodes
    for i, node in enumerate(path):
        x, y = pos[node]
        ax.text(x, y - 0.18, f"#{i+1}", fontsize=9, ha='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='black'))
    
    ax.set_title(f"{title}\nPath: {' → '.join(path)} (Length: {len(path)-1})", 
                fontsize=14, fontweight='bold')
    ax.axis('off')
    plt.tight_layout()
    plt.show()

visualize_path(G, pos, path, "Shortest Path from A to I using BFS")

## 🎯 Enhanced BFS with Path Tracking

Let's implement BFS that not only finds nodes but also tracks the **shortest path** from start to goal.

In [ ]:
def dfs_iterative(graph, start):
    visited = set()
    stack = [start]
    order = []

    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            order.append(node)
            stack.extend(reversed(list(graph.neighbors(node))))
    return order

print("DFS Iterative order:", dfs_iterative(G, 'A'))

# Visualize DFS step-by-step
def dfs_visualization(graph, pos, start):
    """Visualize DFS exploration step by step"""
    visited = set()
    stack = [start]
    order = []
    steps = []  # Store each step for visualization
    
    # Initial state
    steps.append({
        'current': None,
        'stack': list(stack),
        'visited': set(),
        'order': []
    })
    
    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            order.append(node)
            neighbors = list(reversed(list(graph.neighbors(node))))
            unvisited_neighbors = [n for n in neighbors if n not in visited]
            stack.extend(unvisited_neighbors)
            
            # Record this step
            steps.append({
                'current': node,
                'stack': list(stack),
                'visited': visited.copy(),
                'order': order.copy(),
                'neighbors': unvisited_neighbors
            })
    
    # Create visualization
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for idx, step in enumerate(steps[:8]):  # Show first 8 steps
        ax = axes[idx]
        
        # Draw edges
        nx.draw_networkx_edges(graph, pos, edge_color='lightgray', width=2, ax=ax)
        
        # Color nodes based on state
        node_colors = []
        for node in graph.nodes():
            if node == step['current']:
                node_colors.append('#FF6B6B')  # Red = currently processing
            elif node in step['visited']:
                node_colors.append('#4ECDC4')  # Teal = visited
            elif node in step.get('stack', []):
                node_colors.append('#FFD93D')  # Yellow = in stack
            else:
                node_colors.append('#E0E0E0')  # Gray = unvisited
        
        nx.draw_networkx_nodes(graph, pos, node_color=node_colors, 
                              node_size=2000, ax=ax)
        nx.draw_networkx_labels(graph, pos, font_size=10, font_weight='bold', ax=ax)
        
        # Title showing state
        title = f"Step {idx}"
        if step['current']:
            title += f"\nProcessing: {step['current']}"
            title += f"\nStack: {list(step['stack'])}"
        else:
            title += "\nInitial State"
        ax.set_title(title, fontsize=10, fontweight='bold')
        ax.axis('off')
    
    # Create custom legend with colored boxes
    from matplotlib.patches import Rectangle
    legend_elements = [
        Rectangle((0, 0), 1, 1, fc='#FF6B6B', edgecolor='black', label='Current'),
        Rectangle((0, 0), 1, 1, fc='#4ECDC4', edgecolor='black', label='Visited'),
        Rectangle((0, 0), 1, 1, fc='#FFD93D', edgecolor='black', label='In Stack'),
        Rectangle((0, 0), 1, 1, fc='#E0E0E0', edgecolor='black', label='Unvisited')
    ]
    fig.legend(handles=legend_elements, loc='upper center', ncol=4, 
              bbox_to_anchor=(0.5, 0.98), fontsize=12, frameon=True, 
              fancybox=True, shadow=True)
    
    plt.suptitle("DFS Step-by-Step Visualization", 
                 fontsize=14, fontweight='bold', y=0.92)
    plt.tight_layout()
    plt.subplots_adjust(top=0.90)
    plt.show()

dfs_visualization(G, pos, 'A')

## 🎓 Key Takeaways

### When to Use BFS:
- ✅ Finding **shortest path** in unweighted graphs
- ✅ When the goal is likely **close to the start**
- ✅ When you need to explore **level by level**
- ✅ Social network analysis, web crawling

### When to Use DFS:
- ✅ **Memory-constrained** environments
- ✅ When the goal is likely **deep in the tree**
- ✅ Topological sorting, cycle detection
- ✅ Game tree exploration, backtracking problems

### Practical Considerations:
- **BFS** uses more memory (stores all nodes at current level)
- **DFS** can get stuck in infinite loops if not careful
- **BFS** guarantees shortest path (if edges have equal weight)
- **DFS** is easier to implement recursively

In [ ]:
def benchmark_search(algorithm, graph, start, algorithm_name):
    """Benchmark a search algorithm"""
    start_time = time.perf_counter()
    result = algorithm(graph, start)
    end_time = time.perf_counter()
    
    return {
        'name': algorithm_name,
        'nodes_visited': len(result),
        'time_ms': (end_time - start_time) * 1000
    }

# Create different graph types
# 1. Wide graph (many neighbors per node)
wide_graph = nx.Graph()
wide_graph.add_node('root')
for i in range(10):
    wide_graph.add_edge('root', f'L1_{i}')
    for j in range(5):
        wide_graph.add_edge(f'L1_{i}', f'L2_{i}_{j}')

# 2. Deep graph (long chains)
deep_graph = nx.path_graph(50)

# Benchmark
print("📊 Performance Comparison\n")
print("Wide Graph (branching factor = 10):")
bfs_wide = benchmark_search(bfs_basic, wide_graph, 'root', 'BFS')
dfs_wide = benchmark_search(dfs_iterative, wide_graph, 'root', 'DFS')
print(f"  BFS: {bfs_wide['nodes_visited']} nodes, {bfs_wide['time_ms']:.3f}ms")
print(f"  DFS: {dfs_wide['nodes_visited']} nodes, {dfs_wide['time_ms']:.3f}ms")

print("\nDeep Graph (depth = 50):")
bfs_deep = benchmark_search(bfs_basic, deep_graph, 0, 'BFS')
dfs_deep = benchmark_search(dfs_iterative, deep_graph, 0, 'DFS')
print(f"  BFS: {bfs_deep['nodes_visited']} nodes, {bfs_deep['time_ms']:.3f}ms")
print(f"  DFS: {dfs_deep['nodes_visited']} nodes, {dfs_deep['time_ms']:.3f}ms")

# Visualize the different graph structures
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Wide graph visualization
ax1 = axes[0]
pos_wide = nx.spring_layout(wide_graph, seed=42, k=1.5)
nx.draw_networkx_nodes(wide_graph, pos_wide, node_color='lightblue', 
                      node_size=300, ax=ax1)
nx.draw_networkx_edges(wide_graph, pos_wide, edge_color='gray', 
                      width=1, alpha=0.5, ax=ax1)
# Highlight root
nx.draw_networkx_nodes(wide_graph, pos_wide, nodelist=['root'], 
                      node_color='#FF6B6B', node_size=500, ax=ax1)
ax1.set_title(f"Wide Graph Structure\n{bfs_wide['nodes_visited']} nodes, Branching factor ≈ 10\n" +
             f"BFS: {bfs_wide['time_ms']:.3f}ms | DFS: {dfs_wide['time_ms']:.3f}ms", 
             fontsize=12, fontweight='bold')
ax1.axis('off')

# Deep graph visualization
ax2 = axes[1]
pos_deep = nx.spring_layout(deep_graph, seed=42, k=3)
node_colors_deep = ['#FF6B6B' if i == 0 else '#4ECDC4' if i == 49 else 'lightblue' 
                   for i in deep_graph.nodes()]
nx.draw_networkx_nodes(deep_graph, pos_deep, node_color=node_colors_deep, 
                      node_size=200, ax=ax2)
nx.draw_networkx_edges(deep_graph, pos_deep, edge_color='gray', 
                      width=1, alpha=0.5, ax=ax2)
ax2.set_title(f"Deep Graph Structure\n50 nodes in a chain (depth = 50)\n" +
             f"BFS: {bfs_deep['time_ms']:.3f}ms | DFS: {dfs_deep['time_ms']:.3f}ms", 
             fontsize=12, fontweight='bold')
ax2.axis('off')

plt.suptitle("Graph Structure Comparison: Wide vs Deep", 
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Create bar chart comparison
fig, ax = plt.subplots(figsize=(10, 6))

algorithms = ['BFS (Wide)', 'DFS (Wide)', 'BFS (Deep)', 'DFS (Deep)']
times = [bfs_wide['time_ms'], dfs_wide['time_ms'], 
         bfs_deep['time_ms'], dfs_deep['time_ms']]
colors = ['#51CF66', '#4ECDC4', '#51CF66', '#4ECDC4']

bars = ax.bar(algorithms, times, color=colors, edgecolor='black', linewidth=2)

# Add value labels on bars
for bar, time_val in zip(bars, times):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{time_val:.3f}ms', ha='center', va='bottom', fontweight='bold')

ax.set_ylabel('Time (milliseconds)', fontsize=12, fontweight='bold')
ax.set_title('Performance Comparison: BFS vs DFS on Different Graph Structures', 
            fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## ⚡ Performance Comparison

Let's compare the performance characteristics of BFS and DFS on different graph structures.

In [ ]:
# Create a simple maze (0 = path, 1 = wall)
maze = np.array([
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 0, 1, 0]
])

def get_maze_neighbors(pos, maze):
    """Get valid neighboring positions in the maze"""
    rows, cols = maze.shape
    r, c = pos
    neighbors = []
    
    # Check all 4 directions: up, down, left, right
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_r, new_c = r + dr, c + dc
        if 0 <= new_r < rows and 0 <= new_c < cols and maze[new_r, new_c] == 0:
            neighbors.append((new_r, new_c))
    
    return neighbors

def solve_maze_dfs(maze, start, goal):
    """Solve maze using DFS"""
    stack = [(start, [start])]
    visited = set([start])
    
    while stack:
        (pos, path) = stack.pop()
        
        if pos == goal:
            return path
        
        for neighbor in get_maze_neighbors(pos, maze):
            if neighbor not in visited:
                visited.add(neighbor)
                stack.append((neighbor, path + [neighbor]))
    
    return None

def visualize_maze(maze, path=None, title='Maze', visited=None):
    """Visualize the maze and solution path"""
    fig, ax = plt.subplots(figsize=(7, 7))
    
    # Create colored maze
    maze_display = np.ones((maze.shape[0], maze.shape[1], 3))
    
    for i in range(maze.shape[0]):
        for j in range(maze.shape[1]):
            if maze[i, j] == 1:
                maze_display[i, j] = [0.2, 0.2, 0.2]  # Black walls
            else:
                maze_display[i, j] = [1, 1, 1]  # White paths
    
    # Show visited cells in light blue
    if visited:
        for pos in visited:
            if pos not in (path if path else []):
                maze_display[pos] = [0.7, 0.9, 1]  # Light blue
    
    # Show solution path in green
    if path:
        for pos in path:
            maze_display[pos] = [0.6, 1, 0.6]  # Light green
    
    # Display maze
    ax.imshow(maze_display, interpolation='nearest')
    
    # Add grid
    for i in range(maze.shape[0] + 1):
        ax.axhline(i - 0.5, color='gray', linewidth=1)
    for j in range(maze.shape[1] + 1):
        ax.axvline(j - 0.5, color='gray', linewidth=1)
    
    # Mark start and goal with larger markers
    if path:
        start_pos = path[0]
        goal_pos = path[-1]
        ax.plot(start_pos[1], start_pos[0], 'o', color='green', 
               markersize=25, label='Start', markeredgecolor='black', markeredgewidth=2)
        ax.plot(goal_pos[1], goal_pos[0], 'o', color='red', 
               markersize=25, label='Goal', markeredgecolor='black', markeredgewidth=2)
        
        # Draw path with arrows
        for i in range(len(path) - 1):
            y1, x1 = path[i]
            y2, x2 = path[i + 1]
            ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                       arrowprops=dict(arrowstyle='->', lw=3, color='darkgreen'))
        
        ax.legend(loc='upper right', fontsize=12)
    
    ax.set_xticks(range(maze.shape[1]))
    ax.set_yticks(range(maze.shape[0]))
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()

# Solve the maze
start_pos = (0, 0)
goal_pos = (4, 4)

# Show the maze first
print("Original Maze:")
visualize_maze(maze, title='Maze Layout\n⬛ Walls | ⬜ Paths')

# Solve and visualize
solution_path = solve_maze_dfs(maze, start_pos, goal_pos)

if solution_path:
    print(f"\n✅ Path found! Length: {len(solution_path)} steps")
    visualize_maze(maze, solution_path, 
                  title=f'Maze Solution using DFS\nPath length: {len(solution_path)} steps')
else:
    print("❌ No path found!")

# Visualize step-by-step maze solving
def visualize_maze_steps(maze, start, goal):
    """Show how DFS explores the maze step by step"""
    stack = [(start, [start])]
    visited = set([start])
    steps = [{'path': [start], 'visited': {start}, 'current': start}]
    
    while stack and len(steps) < 15:  # Limit steps for visualization
        (pos, path) = stack.pop()
        
        if pos == goal:
            steps.append({'path': path, 'visited': visited.copy(), 'current': pos, 'found': True})
            break
        
        for neighbor in get_maze_neighbors(pos, maze):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = path + [neighbor]
                stack.append((neighbor, new_path))
                steps.append({'path': new_path, 'visited': visited.copy(), 'current': neighbor})
    
    # Visualize key steps
    num_steps = min(6, len(steps))
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    step_indices = [0] + [int(i * (len(steps)-1) / (num_steps-1)) for i in range(1, num_steps)]
    
    for idx, step_idx in enumerate(step_indices):
        ax = axes[idx]
        step = steps[step_idx]
        
        # Create colored maze
        maze_display = np.ones((maze.shape[0], maze.shape[1], 3))
        
        for i in range(maze.shape[0]):
            for j in range(maze.shape[1]):
                if maze[i, j] == 1:
                    maze_display[i, j] = [0.2, 0.2, 0.2]  # Walls
                elif (i, j) in step['visited']:
                    maze_display[i, j] = [0.8, 0.9, 1]  # Visited
                else:
                    maze_display[i, j] = [1, 1, 1]  # Unvisited
        
        # Highlight current path
        for pos in step['path']:
            maze_display[pos] = [0.6, 1, 0.6]  # Current path
        
        # Highlight current position
        maze_display[step['current']] = [1, 0.8, 0]  # Orange
        
        ax.imshow(maze_display, interpolation='nearest')
        
        # Add grid
        for i in range(maze.shape[0] + 1):
            ax.axhline(i - 0.5, color='gray', linewidth=0.5)
        for j in range(maze.shape[1] + 1):
            ax.axvline(j - 0.5, color='gray', linewidth=0.5)
        
        # Mark start and goal
        ax.plot(start[1], start[0], 'go', markersize=15, markeredgecolor='black', markeredgewidth=2)
        ax.plot(goal[1], goal[0], 'ro', markersize=15, markeredgecolor='black', markeredgewidth=2)
        
        ax.set_title(f"Step {step_idx + 1}\nCurrent: {step['current']}", 
                    fontsize=11, fontweight='bold')
        ax.set_xticks(range(maze.shape[1]))
        ax.set_yticks(range(maze.shape[0]))
        ax.tick_params(labelsize=8)
    
    plt.suptitle("DFS Maze Solving: Step-by-Step\n🟩 Current Path | 🟦 Explored | 🟧 Current Position", 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_maze_steps(maze, start_pos, goal_pos)

## 🗺️ Real-World Example: Maze Solving

DFS is commonly used for maze generation and solving. Let's create a simple maze solver.

In [ ]:
# Create a social network graph
social_network = nx.Graph()
friendships = [
    ('Alice', 'Bob'), ('Alice', 'Charlie'), ('Alice', 'David'),
    ('Bob', 'Eve'), ('Bob', 'Frank'),
    ('Charlie', 'Grace'), ('David', 'Henry'),
    ('Eve', 'Ian'), ('Frank', 'Jane'),
    ('Grace', 'Kevin'), ('Henry', 'Linda')
]
social_network.add_edges_from(friendships)

# Visualize the social network
pos_social = nx.spring_layout(social_network, seed=42, k=0.8)
plt.figure(figsize=(10, 7))
nx.draw_networkx_nodes(social_network, pos_social, node_color='lightcoral', node_size=3000)
nx.draw_networkx_edges(social_network, pos_social, edge_color='gray', width=2)
nx.draw_networkx_labels(social_network, pos_social, font_size=10, font_weight='bold')
plt.title("Social Network: Friend Connections", fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

# Find shortest connection between two people
def degrees_of_separation(graph, person1, person2):
    """Find how many connections separate two people"""
    path = bfs_with_path(graph, person1, person2)
    if path:
        degrees = len(path) - 1
        print(f"\n🔗 Degrees of separation between {person1} and {person2}: {degrees}")
        print(f"Connection path: {' → '.join(path)}")
        return degrees, path
    else:
        print(f"No connection found between {person1} and {person2}")
        return None, None

# Visualize the connection path
def visualize_social_path(graph, pos, person1, person2):
    """Visualize the connection path in social network"""
    degrees, path = degrees_of_separation(graph, person1, person2)
    
    if path:
        fig, ax = plt.subplots(figsize=(10, 7))
        
        # Draw all edges in light gray
        nx.draw_networkx_edges(graph, pos, edge_color='lightgray', 
                              width=1, alpha=0.3, ax=ax)
        
        # Highlight path edges
        path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
        nx.draw_networkx_edges(graph, pos, edgelist=path_edges, 
                              edge_color='#FF6B6B', width=5, ax=ax,
                              arrows=True, arrowsize=20, arrowstyle='->')
        
        # Color nodes
        node_colors = []
        for node in graph.nodes():
            if node == person1:
                node_colors.append('#51CF66')  # Green = start
            elif node == person2:
                node_colors.append('#FF6B6B')  # Red = target
            elif node in path:
                node_colors.append('#FFD93D')  # Yellow = on path
            else:
                node_colors.append('#E0E0E0')  # Gray = not on path
        
        nx.draw_networkx_nodes(graph, pos, node_color=node_colors, 
                              node_size=3000, ax=ax, edgecolors='black', linewidths=2)
        nx.draw_networkx_labels(graph, pos, font_size=10, font_weight='bold', ax=ax)
        
        # Add hop numbers
        for i, node in enumerate(path):
            x, y = pos[node]
            ax.text(x, y - 0.15, f"Hop {i}", fontsize=8, ha='center',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', 
                            edgecolor='black', alpha=0.8))
        
        ax.set_title(f"Connection Path: {person1} to {person2}\n" + 
                    f"Path: {' → '.join(path)}\n{degrees} degrees of separation", 
                    fontsize=14, fontweight='bold')
        ax.axis('off')
        plt.tight_layout()
        plt.show()

degrees_of_separation(social_network, 'Alice', 'Linda')
visualize_social_path(social_network, pos_social, 'Alice', 'Linda')

degrees_of_separation(social_network, 'Alice', 'Kevin')
visualize_social_path(social_network, pos_social, 'Alice', 'Kevin')

## 🏗️ Real-World Example: Social Network Analysis

Let's apply BFS to find the "degrees of separation" between people in a social network.

In [ ]:
def visualize_search(graph, pos, order, title):
    """Visualize the order in which nodes are visited"""
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Color nodes based on visit order (gradient)
    colors = []
    for i in range(len(order)):
        # Create gradient from red to blue
        ratio = i / max(len(order) - 1, 1)
        r = int(255 * (1 - ratio))
        g = int(100 * ratio)
        b = int(255 * ratio)
        colors.append(f'#{r:02x}{g:02x}{b:02x}')
    
    node_colors = {node: colors[i] for i, node in enumerate(order)}
    
    # Draw the graph
    nx.draw_networkx_edges(graph, pos, edge_color='gray', width=2, alpha=0.6, ax=ax)
    
    # Draw nodes with colors representing visit order
    for i, node in enumerate(order):
        nx.draw_networkx_nodes(graph, pos, nodelist=[node], 
                              node_color=[node_colors[node]], 
                              node_size=2500, ax=ax, edgecolors='black', linewidths=2)
        # Add visit order number
        x, y = pos[node]
        ax.text(x, y + 0.15, str(i+1), fontsize=10, ha='center', 
                bbox=dict(boxstyle='circle', facecolor='white', edgecolor='black', linewidth=2))
    
    nx.draw_networkx_labels(graph, pos, font_size=14, font_weight='bold', ax=ax)
    
    ax.set_title(f"{title}\nVisit Order: {' → '.join(order)}", 
                fontsize=14, fontweight='bold')
    ax.axis('off')
    
    # Add color bar legend
    ax.text(0.02, 0.98, "Visit Order:\n🔴 First\n⬇️\n🔵 Last", 
           transform=ax.transAxes, fontsize=10, verticalalignment='top',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor='black'))
    
    plt.tight_layout()
    plt.show()

# Visualize BFS
bfs_order = bfs_basic(G, 'A')
visualize_search(G, pos, bfs_order, "BFS Exploration")

# Visualize DFS
dfs_order = dfs_iterative(G, 'A')
visualize_search(G, pos, dfs_order, "DFS Exploration")

# Side-by-side comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# BFS visualization
nx.draw_networkx_edges(G, pos, edge_color='lightgray', width=2, alpha=0.6, ax=ax1)
for i, node in enumerate(bfs_order):
    ratio = i / max(len(bfs_order) - 1, 1)
    color = f'#{int(255*(1-ratio)):02x}{int(100*ratio):02x}{int(255*ratio):02x}'
    nx.draw_networkx_nodes(G, pos, nodelist=[node], node_color=[color], 
                          node_size=2500, ax=ax1, edgecolors='black', linewidths=2)
    x, y = pos[node]
    ax1.text(x, y + 0.15, str(i+1), fontsize=10, ha='center', 
            bbox=dict(boxstyle='circle', facecolor='white', edgecolor='black'))
nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold', ax=ax1)
ax1.set_title(f"BFS (Breadth-First)\nOrder: {' → '.join(bfs_order)}\nLevel-by-level exploration", 
             fontsize=12, fontweight='bold')
ax1.axis('off')

# DFS visualization
nx.draw_networkx_edges(G, pos, edge_color='lightgray', width=2, alpha=0.6, ax=ax2)
for i, node in enumerate(dfs_order):
    ratio = i / max(len(dfs_order) - 1, 1)
    color = f'#{int(255*(1-ratio)):02x}{int(100*ratio):02x}{int(255*ratio):02x}'
    nx.draw_networkx_nodes(G, pos, nodelist=[node], node_color=[color], 
                          node_size=2500, ax=ax2, edgecolors='black', linewidths=2)
    x, y = pos[node]
    ax2.text(x, y + 0.15, str(i+1), fontsize=10, ha='center', 
            bbox=dict(boxstyle='circle', facecolor='white', edgecolor='black'))
nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold', ax=ax2)
ax2.set_title(f"DFS (Depth-First)\nOrder: {' → '.join(dfs_order)}\nGo deep before wide", 
             fontsize=12, fontweight='bold')
ax2.axis('off')

plt.suptitle("Direct Comparison: BFS vs DFS Exploration Patterns", 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 🎨 Visual Comparison: BFS vs DFS

Let's visualize how BFS and DFS explore the graph differently.

In [ ]:
def dfs_recursive(graph, node, visited=None, order=None):
    """Recursive DFS implementation"""
    if visited is None:
        visited = set()
    if order is None:
        order = []
    
    visited.add(node)
    order.append(node)
    
    for neighbor in graph.neighbors(node):
        if neighbor not in visited:
            dfs_recursive(graph, neighbor, visited, order)
    
    return order

print("DFS Recursive order:", dfs_recursive(G, 'A'))

# Visualize recursion call stack
def visualize_recursion_stack(graph, pos, start):
    """Visualize how DFS recursion works with call stack"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Simulate recursive calls
    call_sequences = [
        {'node': 'A', 'stack': ['A'], 'visited': set()},
        {'node': 'B', 'stack': ['A', 'B'], 'visited': {'A'}},
        {'node': 'D', 'stack': ['A', 'B', 'D'], 'visited': {'A', 'B'}},
    ]
    
    for idx, state in enumerate(call_sequences):
        ax = axes[idx]
        
        # Draw graph
        nx.draw_networkx_edges(graph, pos, edge_color='lightgray', width=2, ax=ax)
        
        # Color nodes
        node_colors = []
        for node in graph.nodes():
            if node == state['node']:
                node_colors.append('#FF6B6B')  # Red = current
            elif node in state['visited']:
                node_colors.append('#4ECDC4')  # Teal = visited
            else:
                node_colors.append('#E0E0E0')  # Gray = unvisited
        
        nx.draw_networkx_nodes(graph, pos, node_color=node_colors, 
                              node_size=2000, ax=ax)
        nx.draw_networkx_labels(graph, pos, font_size=12, font_weight='bold', ax=ax)
        
        # Show call stack
        stack_str = '\n'.join([f"dfs({n})" for n in reversed(state['stack'])])
        ax.text(0.02, 0.98, f"Call Stack:\n{stack_str}", 
               transform=ax.transAxes, fontsize=10, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        ax.set_title(f"Recursive Call #{idx+1}\nProcessing: {state['node']}", 
                    fontsize=12, fontweight='bold')
        ax.axis('off')
    
    plt.suptitle("DFS Recursion: Understanding the Call Stack", 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_recursion_stack(G, pos, 'A')

## 🔄 DFS with Recursion

DFS can be elegantly implemented using recursion, which naturally uses the call stack.

## 🧩 Comprehensive Summary

### Algorithm Comparison

| Algorithm | Data Structure | Complete | Optimal | Time Complexity | Space Complexity |
|------------|----------------|-----------|----------|-----------------|------------------|
| **BFS** | Queue (FIFO) | ✅ Yes | ✅ Yes* | O(b<sup>d</sup>) | O(b<sup>d</sup>) |
| **DFS** | Stack (LIFO) | ❌ No** | ❌ No | O(b<sup>m</sup>) | O(bm) |

**Legend:**
- *b* = branching factor (average number of neighbors per node)
- *d* = depth of shallowest goal
- *m* = maximum depth of the search tree
- \* BFS is optimal when all edges have equal cost
- \** DFS is complete in finite spaces without cycles

### 🎯 Decision Guide

**Choose BFS when:**
- You need the **shortest path**
- The solution is **shallow** (near the start)
- Memory is **not a concern**
- Graph is **unweighted** or all edges have equal cost

**Choose DFS when:**
- You need **less memory**
- The solution is **deep** in the tree
- You're doing **backtracking** (e.g., puzzles, games)
- You need to explore **all possible paths**

### 📚 Next Steps
- **Lesson 2:** Informed Search (A*, Best-First Search)
- **Lesson 3:** Adversarial Search (Minimax, Alpha-Beta Pruning)
- **Lesson 4:** Constraint Satisfaction Problems

---

### 📖 Additional Resources
- Russell & Norvig - *Artificial Intelligence: A Modern Approach*
- Cormen et al. - *Introduction to Algorithms*
- [NetworkX Documentation](https://networkx.org/)

**Questions? Reach out during office hours!** 🚀